# NLP Assignment: Generating Trump Tweets with N-Gram Models

In this assignment, you will use n-gram language models (LM) to model tweets (social media statements) from or about the former U.S. president Donald Trump. The goal will then be to generate new tweets, or do autocompletion, in the writing style of Trump's tweets. The tweets have been scraped from the Twitter social media (since then renamed "X").

Before starting this assignment, the appended `NLP_ngram_cheatsheet.ipynb` notebook provides a tutorial on n-grams and LM basics, using the `nltk` package.

Please code the necessary steps in python, and provide answers in Markdown format in this notebook, under the corresponding instructions and questions below.

Please rename your final file `NLP_Assignment_STUDENTID.ipynb` for submission on moodle, and make sure you "run all" with a fresh kernel, so that outputs show correctly and in order in your submission.

**STUDENT ID:** 19-320-563

### Provided Packages

In [240]:
import numpy as np
import pandas as pd
import nltk
nltk.download('popular', quiet=True)

True

### Additional Packages

In [241]:
import preprocessor as p #pip install tweet-preprocessor
import plotly.graph_objects as go
import string as str
import unicodedata
from nltk.tokenize import word_tokenize
import re

## Part 1: Import, inspect and preprocess the text data

- Import the provided dataset, `Trump_tweets.csv`. We are interested in the variable `Tweet_Text`, which gives the content of each tweet. 
- Before tokenizing, start by cleaning the tweets' format. You should at least normalize the different types of apostrophes and quotes (e.g. `` ’, ”, ` ``) to the corresponding ` ' ` or ` " `, remove line breaks `\n` (careful about not "merging" words), and remove multiple spacing. Also make sure urls (e.g. `https://t.co/wPk7QWpK8Z`) are not split into too many meaningless tokens. 
- (Facultative) Feel free to perform additional cleaning steps that you believe will improve the tokenization or the downstream LMs (in which case, briefly explain why).
- Tokenize the `Tweet_Text` corpus into a list of tokenized tweets (documents). The result should be a list of lists containing word-level tokens (e.g. words, punctuation, and other "special words").
- Show the result for the first five tweets of the corpus.

##### Answer

### Import CSV

In [242]:
trump_data = pd.read_csv('Trump_tweets.csv')

trump_data

,Date,Time,Tweet_Text,Type,Media_Type,Hashtags,Tweet_Id,Tweet_Url,twt_favourites_IS_THIS_LIKE_QUESTION_MARK,Retweets,Unnamed: 10,Unnamed: 11
0,16-11-11,15:26:37,Today we express our deepest gratitude to all ...,text,photo,ThankAVet,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,127213,41112,NaN,NaN
1,16-11-11,13:33:35,Busy day planned in New York. Will soon be mak...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,141527,28654,NaN,NaN
2,16-11-11,11:14:20,Love the fact that the small groups of protest...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,183729,50039,NaN,NaN
3,16-11-11,2:19:44,Just had a very open and successful presidenti...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,214001,67010,NaN,NaN
4,16-11-11,2:10:46,A fantastic day in D.C. Met with President Oba...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,178499,36688,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7370,15-07-16,13:10:00,I loved firing goofball atheist Penn @pennjill...,text,NaN,NaN,6.220000e+17,https://twitter.com/realDonaldTrump/status/621...,953,431,NaN,NaN
7371,15-07-16,10:18:31,I hear @pennjillette show on Broadway is terri...,text,NaN,NaN,6.220000e+17,https://twitter.com/realDonaldTrump/status/621...,1175,1086,NaN,NaN
7372,15-07-16,10:10:17,Irrelevant clown @KarlRove sweats and shakes n...,text,NaN,NaN,6.220000e+17,https://twitter.com/realDonaldTrump/status/621...,1494,930,NaN,NaN
7373,15-07-16,9:44:07,"""@HoustonWelder: Donald Trump is one of the se...",text,NaN,NaN,6.220000e+17,https://twitter.com/realDonaldTrump/status/621...,1800,1738,NaN,NaN


In [243]:
trump_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7375 entries, 0 to 7374
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Date                                       7375 non-null   object 
 1   Time                                       7375 non-null   object 
 2   Tweet_Text                                 7375 non-null   object 
 3   Type                                       7375 non-null   object 
 4   Media_Type                                 1225 non-null   object 
 5   Hashtags                                   2031 non-null   object 
 6   Tweet_Id                                   7375 non-null   float64
 7   Tweet_Url                                  7375 non-null   object 
 8   twt_favourites_IS_THIS_LIKE_QUESTION_MARK  7375 non-null   int64  
 9   Retweets                                   7375 non-null   int64  
 10  Unnamed: 10             

### Select *Tweet_Tex* Only

In [244]:
trump_tweet_text = df = pd.DataFrame({'Tweet_Text': trump_data["Tweet_Text"]})


In [245]:
trump_tweet_text

,Tweet_Text
0,Today we express our deepest gratitude to all ...
1,Busy day planned in New York. Will soon be mak...
2,Love the fact that the small groups of protest...
3,Just had a very open and successful presidenti...
4,A fantastic day in D.C. Met with President Oba...
...,...
7370,I loved firing goofball atheist Penn @pennjill...
7371,I hear @pennjillette show on Broadway is terri...
7372,Irrelevant clown @KarlRove sweats and shakes n...
7373,"""@HoustonWelder: Donald Trump is one of the se..."


### Drop NA and Duplicates 

In [246]:
trump_tweet_text = trump_tweet_text.dropna()
trump_tweet_text = trump_tweet_text.drop_duplicates()
rowb = trump_tweet_text.count()
trump_tweet_text

,Tweet_Text
0,Today we express our deepest gratitude to all ...
1,Busy day planned in New York. Will soon be mak...
2,Love the fact that the small groups of protest...
3,Just had a very open and successful presidenti...
4,A fantastic day in D.C. Met with President Oba...
...,...
7370,I loved firing goofball atheist Penn @pennjill...
7371,I hear @pennjillette show on Broadway is terri...
7372,Irrelevant clown @KarlRove sweats and shakes n...
7373,"""@HoustonWelder: Donald Trump is one of the se..."


### Check Samples Tweets

#### RT (Retweet)

> Don't contains any Trump own says

In [247]:
sample1 = trump_tweet_text["Tweet_Text"].iloc[8]
sample1

'RT @IvankaTrump: Such a surreal moment to vote for my father for President of the United States! Make your voice heard and vote! #Election2_'

#### QT (Quotations Tweet)

> May contains some original Trump says, mainly comments added after a quote of someone else

In [248]:
sample2 = trump_tweet_text["Tweet_Text"].iloc[-2]
sample2

'"@HoustonWelder: Donald Trump is one of the sexiest men on this planet. Every woman dreams of a good man who tells it like it is." So true!'

> Here original Trump says is "So true !"

### Tweets Types

In [249]:
trump_tweet_text['Type'] = np.where(trump_tweet_text["Tweet_Text"].str.startswith('RT'), 'RETWEET', 
                       np.where(trump_tweet_text["Tweet_Text"].str.startswith('"@'), 'QUOTATION RETWEET', 'ORIGINAL'))

trump_tweet_text["Type"].value_counts()

ORIGINAL             4809
QUOTATION RETWEET    2125
RETWEET               430
Name: Type, dtype: int64

### Samples Types

In [250]:
sample1 = trump_tweet_text.iloc[8]

sample2 = trump_tweet_text.iloc[-2]

print("\n SAMPLE 1:", sample1[0], """

""","TYPE:",sample1[1], """

---------------------------------------

""","SAMPLE 2:", sample2[0], """

""","TYPE:",sample2[1])


 SAMPLE 1: RT @IvankaTrump: Such a surreal moment to vote for my father for President of the United States! Make your voice heard and vote! #Election2_ 

 TYPE: RETWEET 

---------------------------------------

 SAMPLE 2: "@HoustonWelder: Donald Trump is one of the sexiest men on this planet. Every woman dreams of a good man who tells it like it is." So true! 

 TYPE: QUOTATION RETWEET


### Removing Retweet and keeping Quotation Retweet

> I think that keeping Retweets will not be relevant in creating Trump Tweets style, at most it will generate tweets that he likes or think need to be spread, but worse it can attribute the words from other to his own speech without being relevant to how he talks or think. Keeping Quotation Retweets is different than simple Retweets, in the sense that Trumps usually likes to comments or criticise over existing tweets, which may help the model to grasp this kind of behavior. 

In [251]:
trump_tweet_text = trump_tweet_text[trump_tweet_text['Type'] != 'RETWEET']
trump_tweet_text

,Tweet_Text,Type
0,Today we express our deepest gratitude to all ...,ORIGINAL
1,Busy day planned in New York. Will soon be mak...,ORIGINAL
2,Love the fact that the small groups of protest...,ORIGINAL
3,Just had a very open and successful presidenti...,ORIGINAL
4,A fantastic day in D.C. Met with President Oba...,ORIGINAL
...,...,...
7369,I hope the boycott of @Macys continues forever...,ORIGINAL
7370,I loved firing goofball atheist Penn @pennjill...,ORIGINAL
7371,I hear @pennjillette show on Broadway is terri...,ORIGINAL
7372,Irrelevant clown @KarlRove sweats and shakes n...,ORIGINAL


### Preprocess

In [308]:
# Define a function to clean the tweets
def clean_tweet(tweet):
    
    # Normalize apostrophes and quotes
    tweet = unicodedata.normalize('NFKD', tweet).encode('ascii', 'ignore').decode('utf-8')
    
    # Remove line breaks
    tweet = tweet.replace('\n', ' ')
    
    # Remove multiple spacing
    tweet = re.sub(r'\s+', ' ', tweet)
    
    # Remove URLs
    #tweet = re.sub(r'https?://\S+', '', tweet)
    
    # Remove special characters (optional)
    #tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    
    return tweet

In [309]:
# Apply the cleaning function to the Tweet_Text column
trump_tweet_text['Tweet_Text_Clean'] = trump_tweet_text['Tweet_Text'].apply(clean_tweet)

/var/folders/j0/m71dtf9j0rvddgqlps9swgxc0000gn/T/ipykernel_12154/4020429102.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [310]:
row = 23
print(trump_tweet_text.iloc[row][0], "\n\n",trump_tweet_text.iloc[row][2])

America must decide between failed policies or fresh perspective, a corrupt system or an outsider
https://t.co/ll8QIW9SqW 

 America must decide between failed policies or fresh perspective, a corrupt system or an outsider https://t.co/ll8QIW9SqW


### Tokenization

> Testing Tokens with Casual from NLTK

In [319]:
text = trump_tweet_text.iloc[800][2]
 
text_tokens = nltk.casual_tokenize(text)
text_tokens

['Thank', 'you', 'Ohio', '!', '#AmericaFirst', 'https://t.co/p68GAJdhwu']

## Part 2: Fitting and Accessing a Trump Tweet LM

### Ex. 2.1: LM fitting function
Create a function that takes as arguments (at least) the desired order $n$ of the model and a tokenized training corpus, and that returns the "simple" Maximum Likelihood Estimator (MLE) language model, fitted on the given training corpus.  
Then, use your function to fit a MLE language model of order $n=3$ to the Trump Tweets corpus.

##### Answer

### Ex. 2.2: Vocabulary
- How many distinct tokens are in the model's vocabulary? Is that the same number of distinct tokens that appear in the tokenized corpus?
- Lookup the tokens of the sentence `"I love UNIGE students!"` in the model's vocabulary. Explain what you observe, and why. 

##### Answer

### Ex. 2.3: Token probabilities
- When it comes to ngram models the training boils down to counting the ngrams from the training corpus. Using your fitted model, how many times do the following appear in the training data: ``'America', 'Trump', 'I will', 'will never forget'``.
- Then, compute the following word occurrence probabilities ('scores') in the Trump Tweets corpus, and briefly explain what the returned numbers mean about the training data:
    - $\mathbb{P}($'America'$)$,
    - $\mathbb{P}($'Trump'$)$,
    - $\mathbb{P}($'will'$\vert $'I'$)$,
    - $\mathbb{P}($'forget'$\vert $'will never'$)$.
- Briefly explain, with a formula, how those probabilities are obtained from the n-gram counts.

##### Answer

## Part 3: Generation using N-gram Language Model

### Ex. 3.1: Tweet generator
Create a python function to generate new Trump Tweets. It should:
- take as input arguments: a fitted `nltk.lm.model`, a maximum number of words (integer), a text seed (initial context tokens), and a random "RNG" seed for generation,
- output a newly generated Trump Tweet, according to the input arguments, post-processed as a single text string that is formatted like a tweet.

*Hints:* `nltk.tokenize.treebank.TreebankWordDetokenizer()` and its `.detokenize()` method can help with post-processing. Pay attention to show things like `@user` mentions, urls, punctuation, etc... in a "correct" format.

##### Answer

### Ex. 3.2: Initial context
To generate a full tweet from a LM of order $n$, explain what should be the text seed (i.e. the initial context tokens). Set the default value for the relevant argument of your function in 3.1 accordingly.

##### Answer

### Ex. 3.3: Generate tweets
Generate a few new tweets using your new function and the LM fitted in Part 2. For reproducibility, use a random RNG seed to show them. 

*Facultative:* show a few examples that you find interesting, representative or funny.

##### Answer

## Part 4: Smoothing and model comparison

### Ex. 4.1: Smoothed LM alternatives to simple MLE
Modify the function that you defined in 2.1 by adding an argument that allows changing the `nltk.lm` language model that is fitted in the function (e.g. to fit a Laplace or a Lidstone model instead of the simple MLE). 
Also briefly explain what is the difference between Laplace, Lidstone and the simple MLE language models.

*Hint:* Your function might need more than a single additional argument, if some LM have hyperparameters.

##### Answer

### Ex. 4.2: Qualitative model comparison 
With $n=1,2,3,4$, fit and generate new tweets from the simple MLE and from the Laplace LM of orders $n$. 
- Compare the results between the different $n$ values and between the two models. 
- What are the main differences for generation? Which model(s) do you think might be the best options for generating new realistic tweets?
- Do you see hints of those differences in the generated tweets?

##### Answer

### Ex. 4.3: Quantitative evaluation and comparison
- Split the tokenized Trump Tweets corpus into a (reproducible) training set (80%) and a test set (20%). 
- Compute the train and test 3-gram perplexity scores of a simple MLE LM, a Laplace LM, and a Lidstone LM with $\gamma=0.1$. Use model order $n=3$ for each.
- Compare and discuss the obtained train and test perplexity scores of the three models. Argue which model might represent the Trump Tweets data best.

*Hint:* To compute the perplexity correctly, you might need to preprocess the relevant corpus documents to a list of padded $n$-grams.

##### Answer

### Ex. 4.4: Hyper-parameter tuning
- Perform a grid-search to select the best hyperparameter values for $n$ and $\gamma$, for the Lidstone LM. You want to select the model that generalizes best to new data.
- What do you observe in the obtained perplexity scores? Was it expected? Explain it in statistical terms.

*Hint:* Maybe try a few values for $n$ and $\gamma$ by hand to identify the general hyperparameter region of interest before defining a more thorough hyperparameter value grid.

##### Answer